# Imports

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import entropy
from scipy.fft import fft, fftfreq
from sklearn.decomposition import FastICA
from scipy.signal import butter, lfilter, correlate, detrend

### Methods

In [1]:
# Function to apply bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y

# Function to extract RGB means from ROI
def extract_rgb_means(frame, roi):
    r = np.mean(frame[roi[1]:roi[3], roi[0]:roi[2], 0])
    g = np.mean(frame[roi[1]:roi[3], roi[0]:roi[2], 1])
    b = np.mean(frame[roi[1]:roi[3], roi[0]:roi[2], 2])
    # print('r', r)
    # print('g', g)
    # print('b', b)
    return r, g, b

# Function to calculate negentropy
def calculate_negentropy(signal):
    # Ensure no NaN or Inf values in signal
    signal = np.nan_to_num(signal)
    if np.isnan(signal).any() or np.isinf(signal).any():
        print("Signal contains NaNs or Infs, returning NaN for negentropy.")
        return np.nan

    # Normalize the signal
    signal_norm = (signal - np.mean(signal)) / np.std(signal)
    gaussian = np.random.normal(0, 1, len(signal))
    return entropy(gaussian) - entropy(signal_norm)

# Function to calculate autocorrelation
def calculate_autocorrelation(signal):
    result = correlate(signal, signal, mode='full')
    max_corr = np.max(result[result.size // 2:])
    return max_corr / len(signal)

# MAICA method for multi-objective optimization
def maica(ica_signals, fs):
    best_signal = None
    best_score = -np.inf

    for i in range(ica_signals.shape[1]):
        signal = ica_signals[:, i]
        filtered_signal = bandpass_filter(signal, 0.7, 4.0, fs)
        
        # Calculate metrics
        autocorr = calculate_autocorrelation(filtered_signal)
        negentropy = calculate_negentropy(filtered_signal)
        
        if np.isnan(negentropy):
            print(f"Skipping component {i} due to NaN in negentropy calculation.")
            continue

        # Combine metrics (this is a simple weighted sum, adjust weights as needed)
        score = autocorr + negentropy
        
        if score > best_score:
            best_score = score
            best_signal = filtered_signal

    return best_signal

def checkForInfsOrNaNs(rgb_signals):
    # Check for NaNs or Infs in the data
    if np.any(np.isnan(rgb_signals)) or np.any(np.isinf(rgb_signals)):
        print("Data contains NaNs or Infs, handling them...")
        # # Option 1: Remove rows with NaN or Inf values
        # rgb_signals = rgb_signals[~np.isnan(rgb_signals).any(axis=1)]
        # rgb_signals = rgb_signals[~np.isinf(rgb_signals).any(axis=1)]
    
        # # Option 2: Replace NaNs with zeros (or another appropriate value)
        # rgb_signals = np.nan_to_num(rgb_signals)
    
        # Option 3: Interpolate or fill NaNs/Infs if possible
        # Example: fill NaNs with the mean of the column
        rgb_signals = np.where(np.isnan(rgb_signals), np.nanmean(rgb_signals, axis=0), rgb_signals)
    
    # Ensure rgb_signals has no NaNs or Infs now
    assert not np.any(np.isnan(rgb_signals)), "NaNs still present in the data"
    assert not np.any(np.isinf(rgb_signals)), "Infs still present in the data"
    
    return rgb_signals

## Video

In [3]:
vidPath = r"C:\Users\USER\Documents\SLIIT\Datasets\UBFC_DATASET\DATASET_1\5-gt\vid.avi"

In [4]:
# Capture video
cap = cv2.VideoCapture(vidPath)
fs = cap.get(cv2.CAP_PROP_FPS)  # Frame rate
roi = (170, 320, 370, 480)  # Example ROI, adjust as needed
rgb_signals = []

while True:
    ret, frame = cap.read()
    # plt.imshow(frame[170:370, 320:480])
    # cv2.waitKey(1)
    
    # print(frame.shape)
    # break
    
    if not ret:
        break

    r, g, b = extract_rgb_means(frame, roi)
    rgb_signals.append([r, g, b])
    
    cv2.rectangle(frame, (170, 320), (370,480), (255, 0, 0), 2)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

### RPPG Signal extraction

In [5]:
rgb_signals = np.array(rgb_signals)

# Detrend and normalize
# rgb_signals = checkForInfsOrNaNs(rgb_signals)
rgb_signals = detrend(rgb_signals, axis=0)
rgb_signals = (rgb_signals - np.mean(rgb_signals, axis=0)) / np.std(rgb_signals, axis=0)

# Apply ICA
ica = FastICA(n_components=3)
ica_signals = ica.fit_transform(rgb_signals)

In [6]:
# Apply MAICA
rppg_signal = maica(ica_signals, fs)

Skipping component 0 due to NaN in negentropy calculation.
Skipping component 1 due to NaN in negentropy calculation.
Skipping component 2 due to NaN in negentropy calculation.


C:\Users\USER\AppData\Local\Temp\ipykernel_19156\1024867438.py:31: RuntimeWarning: invalid value encountered in scalar subtract
  return entropy(gaussian) - entropy(signal_norm)


In [6]:
# Estimate heart rate using FFT
n = len(rppg_signal)
t = np.arange(n) / fs
yf = fft(rppg_signal)
xf = fftfreq(n, 1 / fs)[:n // 2]

# Find the peak frequency and convert to bpm
peak_freq = xf[np.argmax(np.abs(yf[:n // 2]))]
heart_rate = peak_freq * 60  # Convert to bpm
print(f"Estimated Heart Rate: {heart_rate:.2f} bpm")

# Plot the rPPG signal and FFT
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(t, rppg_signal)
plt.title('rPPG Signal')
plt.subplot(2, 1, 2)
plt.plot(xf, 2.0 / n * np.abs(yf[:n // 2]))
plt.title('FFT of rPPG Signal')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.show()

C:\Users\USER\AppData\Local\Temp\ipykernel_6128\4163671609.py:24: RuntimeWarning: invalid value encountered in scalar subtract
  return entropy(gaussian) - entropy(signal_norm)


TypeError: object of type 'NoneType' has no len()

None
